In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

with open('dataset/data/train_set.csv') as f:
    train = f.readlines()[1:]
with open('dataset/data/test_b.csv') as f:
    test = f.readlines()[1:]
    x_test = []
    for s in test:
        x_test.append(' '.join(s.strip().split()[::-1]))
#         print(' '.join(s.strip().split()[::-1]))
#         print(s.strip())
        
x, y = [], []
for line in train:
    y_, x_ = line.split('\t')
    x.append(' '.join(x_.strip().split()[::-1]))
    y.append(y_.strip())
    
x_train, x_dev, y_train, y_dev = train_test_split(x, y, test_size=0.2, random_state=0)

In [17]:
with open('dataset/data/test_b.csv') as f:
    test = f.readlines()[1:]
    x_test = []
    for s in test:
        x_test.append(s.strip())

In [2]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(ngram_range=(1,4), min_df=2, max_df=0.95, use_idf=1, smooth_idf=1, sublinear_tf=1)

# vect.fit(x_train+x_dev+x_test)
# pickle.dump(vect, open("dataset/tfidf_vect_2_0.95.pickle", "wb"))
vect = pickle.load(open("dataset/tfidf_vect_2_0.95.pickle", "rb"))

In [18]:
x_

<200000x32543782 sparse matrix of type '<class 'numpy.float64'>'
	with 432115742 stored elements in Compressed Sparse Row format>

In [11]:
# x_train_ = vect.transform(x_train[:])
# x_dev_ = vect.transform(x_dev[:])
x_test_ = vect.transform(x_test)

In [12]:
%%time
# x_train_ = vect.transform(x_train[:])
# x_dev_ = vect.transform(x_dev[:])
x_ = vect.transform(x)

CPU times: user 20min 4s, sys: 46.6 s, total: 20min 51s
Wall time: 21min 6s


In [13]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import f1_score

lsvc = svm.LinearSVC() #0.94516左右
sgdc = SGDClassifier(loss="modified_huber", alpha=0.000008, n_jobs=-1) #0.9447左右
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1) #0.71 0.79
lr = LogisticRegression()

clf = sgdc

k = None
# clf.fit(x_train_[:k], y_train[:k])
clf.fit(x_[:k], y[:k])
print(f1_score(clf.predict(x_), y, average='macro'))

pred = clf.predict(x_dev_)
dev_f1 = f1_score(pred, y_dev, average='macro')
print(dev_f1)

/home/lihaoyuan/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/home/lihaoyuan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.9965681960928319


NameError: name 'x_dev_' is not defined

In [15]:
%%time
clf = svm.LinearSVC()
clf.fit(x_, y)
tra_f1 = f1_score(clf.predict(x_), y, average='macro')
print(tra_f1)

0.9982031213683663
CPU times: user 6min 40s, sys: 40.9 s, total: 7min 21s
Wall time: 7min 23s


In [8]:
# x_test_ = vect.transform(x_test)

In [16]:
try:
    dev_f1 += 0
except:
    dev_f1 = 0.0
    
model_name = 'TFIDF+LSVC_back'
test_pred = clf.predict(x_test_)

with open('results/%s_result_%.4f.csv'%(model_name, dev_f1), 'w') as f:
    f.write('label\n')
    for p in test_pred:
        f.write(str(p)+'\n')
        
try:
    prob = clf.predict_proba(x_test_)
    np.save('results/%s_prob_hard'%model_name, prob)
except:
    pred_ = clf.predict(x_test_).astype('int')
    prob = np.zeros((len(test), 14))
    for i,j in enumerate(pred_):
        prob[i,j] = 1
    np.save('results/%s_prob'%model_name, prob)

In [27]:
print(clf.coef_.T.shape)
prob_ = np.dot(x_test_, clf.coef_.T)

(32543782, 14)


KeyboardInterrupt: 

In [53]:
prob = prob.astype('int')
print(prob)

p = np.zeros((len(test), 14))
for i,j in enumerate(prob):
    p[i,j] = 1
p

[1 2 8 ... 1 3 1]


array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [ ]:
import lightgbm as lgb

train_data = lgb.Dataset(x_train_, label=y_train)
val_data = lgb.Dataset(x_dev_, label=y_dev)

params = {'boosting_type': 'gbdt',
          'num_leaves': 28,
          'max_depth': -1,
          'objective': 'regression',
          'learning_rate': 0.1,
          'seed': 2018,
          'num_threads': -1,
          'max_bin': 425,
          "metric": "rmse",
          # "lambda_l1": 0.1,
          "lambda_l2": 0.2,
          }

params = {'num_threads': -1,
          'objective': 'multiclass',
          'metric':{'multi_logloss', 'multi_error'},
          ''
         }
clf = lgb.train(params, train_set=train_data, num_boost_round=50000, valid_sets=[train_data, val_data],
                  valid_names=['train', 'valid'], early_stopping_rounds=50, feval=None, verbose_eval=50)


dev_f1 = f1_score(clf.predict(x_dev_), y_dev, average='macro')
print(dev_f1)